# Visualization of COVID-19 Hotspots in the US

The purpose of this notebook is to identify COVID-19 "hotspots" in the 50 United States. Data is displayed at the county level.

**<p style="color:blue">Once the interactive version of the notebook appears, wait for a map to appear at the bottom of the notebook.  JavaScript needs to be enabled in your browser for the map to work.</p>**

<p style="color:red">Please note that I have further decreased the default "Red Rate" to 1.05 (5% increase per day). </p>

The current **number** of confirmed cases is shown by the **area** of the circles. Since recoveries/deaths are not accounted for, it is expected that the number of cases in a county will stay the same or increase over time.

The **daily rate of increase** is shown by the **color** of the circles.  The rate of increase for a day is calculated by dividing the number of cases on that day by the number of cases on the previous day.  A five day weighted average is used to smooth out abrupt changes in the data, with the most recent day being weighted most heavily.

A daily rate of increase of 1.0 indicates no change in the number of confirmed cases in the last five days, and is displayed as a green circle.  The rate of increase which is displayed as red ("Red Rate") is selectable by using a slider (the initial value is 1.05), with rates higher than the "Red Rate" also being displayed as red. 

This notebook uses confirmed case data from Johns Hopkins University.  The data is updated daily (usually late at night), so the numbers of cases may be up to one day out of date. Since 5-day averages are being displayed, the potential out-of-datedness of data should not make much difference in identifying hotspots, but if the absolutely latest data is desired, please refer to this visualization:

https://coronavirus.jhu.edu/map.html

To send suggestions for this visualization, please click here:
<A HREF="mailto:covidhotspot@elegambda.com">covidhotspot@elegambda.com</A>. 

If you are interested in seeing the Python code that makes this visualization work, click on the map or the whitespace above the map, then click the **^** button on the toolbar.  Keep doing this until cells 1 through 5 are revealed. You can experiment with the code and rerun the cell you changed and those below it. If you mess something up, just shut down this notebook and click on the original link that started this notebook.

The initial startup can take a minute or so, because each time you click on the link to start this notebook, a personal notebook server is started up.  It is automatically shutdown after a period of inactivity (10 minutes).

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

In [ ]:
from math import sqrt, pow
import numpy as np
import pandas as pd
import folium
from ipywidgets import interact, widgets
import branca.colormap as cm
import us

In [ ]:
# read in the data from JHU github
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
df = pd.read_csv(url,index_col=0)

# filter out all but county-level data
df = df[((df['FIPS'] > 1000) & (df['FIPS'] < 60000))]

# select useful columns and rename them, replacing state names with abbreviations
df_meta = df.iloc[:, [3, 4, 5, 7, 8]]  # all rows, selected columns
df_date = df.iloc[:,11:]               # all rows, columns 11 to the end
df = pd.merge(df_meta, df_date, on='UID')
df.rename(columns = {'Admin2':'County', 'Province_State':'State', 'Long_':'Lon'}, inplace = True)
df.State = [us.states.lookup(name).abbr for name in df.State]  # abbreviate state name

In [ ]:
# define a weighted average rate of increase function
np.seterr(all='ignore');  # suppress divide by zero error messages
def rate_of_increase(row, days=5):
    
    # set up weights
    weights = np.arange(days + 1, dtype=np.float)
    weights = weights / np.sum(weights)

    # calculate daily rates of increase (with errors)
    vals = row[-days-1:].to_numpy(dtype='float64')
    rates = vals[-days:]/vals[-days-1:-1]
      
    # correct for divide by zeros
    rates[np.isnan(rates)] = 0.0    # replace 0/0 with 0.0
    rates[rates==np.inf] = 2.0      # replace first non-zero/zero by large rate
 
    # multiply weights by rates; sum result; return
    rate = np.sum(weights[-days:]*rates)
    return rate

In [ ]:
# create a map
def create_map(red_rate=1.05):
    days_avg=5
    m = folium.Map(location=[39.758056, -96.00000], tiles='CartoDB positron', zoom_start=4, prefer_canvas=True)
    
    # sort rows so that smaller circles are on top
    rows = sorted([rowtuple[1] for rowtuple in df.iterrows() if rowtuple[1][-1] > 0], 
                  key=lambda row: row[-1], reverse=True)

    color_map = cm.LinearColormap(['green', 'red'], vmin=1.0, vmax=red_rate)
    
    # add circles to the map
    for row in rows:
        rate_color=color_map(rate_of_increase(row, days_avg))
        c = folium.Circle([row['Lat'], row['Lon']],
    
                            radius=pow(float(row[-1]), .43)*1000,
    
                            # stroke
                            stroke=True, weight=1, color=rate_color, opacity=0.5,
                      
                            # fill
                            fill=True, fillColor=rate_color, fillOpacity=0.002,
                      
                            # tooltip
                            tooltip = folium.Tooltip(f"<pre>{row['State']}, {row['County']}:<br>  Cases: {row[-1]} <br>  Rate: {rate_of_increase(row, days_avg):.3f}</pre>")                                      
            
                         ).add_to(m)
    return m                          

interact(create_map, red_rate=widgets.FloatSlider(min=1.0, max=1.5, step=0.01, value=1.05,
                                                 description="Red Rate", continuous_update=False));
                          